<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Peter Br&ouml;cker</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Tutorial:   Animationen als Video speichern</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2016</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml)

# Einleitung

Auf den vorherigen Übungsblättern haben Sie eine Reihe von Aufgaben bearbeitet, in denen wir  zeitabhängige Phänomene betrachten. Naturgemäß möchten wir in diesem Fall die Ergebnisse der Iteration zeitaufgelöst darstellen. In diesem Notebook demonstrieren wir, wie man eine Sequenz von Bildern, die ebendiese Iterationsschritte repräsentieren, in ein Video umwandelt.

Dazu benutzen wir das Programm *ffmpeg*, das Sie für [Linux](http://ffmpeg.org/download.html#build-linux), [Windows](http://ffmpeg.org/download.html#build-windows) und [MacOSX](http://ffmpeg.org/download.html#build-mac) von der [offiziellen Homepage](http://ffmpeg.org/) herunterladen können. 

Unter **Linux** besteht die Möglichkeit, dass Sie das Programm direkt über Ihre Paketverwaltung installieren können. Unter **MacOS X** müssen sie lediglich das *zip*-Archiv entpacken. Unter **Windows** müssen sie wahrscheinlich zusätzlich das Programm [7-Zip](http://www.7-zip.org/) herunterladen, um das Archiv zu entpacken. 

# Installation


Unter Linux und MacOS X kann die gesamte Installation über dieses Notebook erfolgen, wie wir nun erläutern. Wir erstellen dazu in Ihrem privaten Nutzerverzeichnis einen Unterordner *.local*, in den wir die aktuelle Version von *ffmpeg* herunterladen und entpacken.

Unter Windows können wir lediglich die derzeit aktuellen Links angeben und ausführen, um die Installation zu starten.

Falls Sie *ffmpeg* bereits auf anderem Wege installiert haben, können Sie diesen Teil überspringen.

In [ ]:
cd(ENV["HOME"])
if !isdir(".local") mkdir(".local") end
cd(".local")
println("Installiere 'ffmpeg' im Ordner ", pwd())

# os = 0    Linux
# os = 1    MacOS X
# os = 2    Windows

os = @linux? 0 : @osx? 1 : 2

path_to_ffmpeg = ""

if os == 0
    download("http://johnvansickle.com/ffmpeg/builds/ffmpeg-git-64bit-static.tar.xz", "ffmpeg.tar.xz") 
    run(`tar xfv ffmpeg.tar.xz`)
    ffmpeg_file = split(readall(`find . -name ffmpeg`))[1][2:end]
    path_to_ffmpeg = pwd() * ffmpeg_file
    
    println("Benutzen Sie von nun diese Variable: path_to_ffmpeg = ", path_to_ffmpeg)
end

if os == 1
    if !isfile("ffmpeg")
        download("http://www.ffmpegmac.net/resources/SnowLeopard_Lion_Mountain_Lion_Mavericks_Yosemite_El-Captain_02.05.2016.zip", "ffmpeg.zip")
        run(pipeline(`unzip ffmpeg.zip`))    
    end
    path_to_ffmpeg = pwd() * "/ffmpeg"
    
    println("Benutzen Sie von nun diese Variable: path_to_ffmpeg = ", path_to_ffmpeg)
end

if os == 2
    download("http://7-zip.org/a/7z1600-x64.exe", "7zip.exe")
    run(pipeline(`7zip.exe`))
    download("https://ffmpeg.zeranoe.com/builds/win64/static/ffmpeg-20160531-git-a1953d4-win64-static.7z", "ffmpeg.7z")
end   



# Beispieldaten

Wir generieren nun Beispieldaten und speichern nach jedem Iterationsschritt ein Bild. Anstatt jedes mal ein neues Bild zu zeichnen, aktualisieren wir die Grafik mit dem Befehl *set_data*.

In [ ]:
using PyPlot

ts = 0:0.01:4
xs, ys = 0.7*cos(ts*pi), 0.7*sin(ts*pi)

# deaktiviere die Anzeige des letzten Plots
ioff()

fig, ax = subplots(figsize=(6, 6))

# Zeichne eine Linie ...
line = ax[:plot]([0, xs[1]], [0, ys[1]], "-", color="black")[1]

# ... und einen Punkt
dot = ax[:plot](xs[1], ys[1], "o", markeredgecolor="none", markerfacecolor="#850000", markersize=20)[1]

# Ploteinstellungen
xlim([-1, 1])
ylim([-1, 1])
axis("off")

for i in 1:length(xs)
    # aktualisiere Bilddaten
    line[:set_data]([0, xs[i]], [0, ys[i]])
    dot[:set_data]([xs[i]], [ys[i]])
    
    # konvertiere Index in String m i t führenden Nullen
    # wichtig für Ordnung im Dateisystem
    a = @sprintf "%04d" i
    savefig("movie_$(a).png")
end
close(fig)
ion()
    

# Erzeugung des Videos


Der Pfad zur ausführbaren Datei ffmpeg(.exe für Windowsnutzer) muss in der Variable *path_to_ffmpeg* gespeichert sein. Wir übergeben nun die folgenden Argumente:

- *start_number* - Startindex
- *framerate* - Bilder pro Sekunde
- *i* - Dateinamenmuster
- *vcodec* - zu verwendender Videocodec, in diesem Fall mp4
- *s* - Videogröße
- *pix_fmt* - Bildformat für das Video

Je nach Situation macht es Sinn die *framerate* und die Bildgröße über *s* anzupassen.

In [ ]:
if isfile("test_movie.mp4")
    rm("test_movie.mp4")
end

run(`$(path_to_ffmpeg) -start_number 1 -framerate 25 -i movie_%04d.png -vcodec libx264 -s 640x640 -pix_fmt yuv420p test_movie.mp4`);

for i in 1:length(xs)
    a = @sprintf "%04d" i
    rm("movie_$(a).png")
end

# Anzeige des Videos

Wir benutzen nun die Funktion *display* des Notebooks, mit dem wir HTML Code in den Ausgabezellen darstellen können, um ein mp4 zu laden und direkt vom Browser abspielen zu lassen.

In [ ]:
base64_video = base64encode(open(readbytes,"test_movie.mp4"))
display("text/html", string("""<video autoplay controls><source src="data:video/x-m4v;base64,""",
base64_video,"""" type="video/mp4"></video>"""));